In [16]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [2]:
with open('../config.json') as file:
    app_data = json.load(file)

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials( \
                     client_id=app_data['client_id'], \
                     client_secret=app_data['client_secret']))

In [3]:
all_years = pd.read_csv('../data/Billboard_Audio_Features.csv')
all_years.head(1)

,Album ID,Artist IDs,Artist(s),ID,Popularity,Query,Rank,Release Date,Song,Year,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,3bbdZdZQtCtnWEN5zG7Eyj,['3Z02hBLubJxuFJfhacLSDc'],Bryan Adams,5HQ6AUDMbMuwktvcNPuV1y,36.0,track:(Everything I Do) I Do It for You artist...,1.0,1991-01-01,(Everything I Do) I Do It for You,1991.0,...,-13.452,1,0.0289,0.0648,0.000013,0.055,0.273,131.189,398027,4


In [59]:
all_years = pd.read_csv('../data/Billboard_Audio_Features.csv')
all_years['Artist IDs'] = all_years['Artist IDs'].str.lstrip('[').str.rstrip(']')
all_years['Primary Artist ID'] = all_years['Artist IDs'].str.split(',', expand = True)[0].str.strip("'")
all_years['Genres'] = pd.Series(dtype=object)
all_with_genres = pd.DataFrame(columns=all_years.columns)

In [65]:
all_years = all_years.tail(3408)
for i, row in all_years.iterrows():
    artist = sp.artist(row['Primary Artist ID'])
    row['Genres'] = ", ".join(artist["genres"])
    all_with_genres = all_with_genres.append(row, ignore_index= True)

In [67]:
all_with_genres.to_csv('../data/Billboard_Audio_Features.csv', index=False)

In [18]:
data = pd.read_csv('../data/Billboard_Audio_Features.csv')
data['Genres'] = data['Genres'].replace(np.nan, "")

In [20]:
# create dictionary of word frequency in genre
categories = {}
for genre in data['Genres']:
    genre = genre.replace(",", "")
    words = genre.split(' ')
    uniques = []
    for word in words:
        if word not in uniques:
            uniques.append(word)
    for word in uniques:
        if word in categories:
            categories[word] += 1
        else:
            categories[word] = 1

# sort dictionary by frequency to get most common overall genre categories
list({k: v for k, v in sorted(categories.items(), key=lambda item: item[1], reverse=True)}.items())[:15]

[('pop', 3542),
 ('rock', 2367),
 ('classic', 1332),
 ('dance', 1283),
 ('mellow', 1248),
 ('soft', 1168),
 ('gold', 1095),
 ('hip', 1030),
 ('rap', 1006),
 ('contemporary', 990),
 ('new', 885),
 ('soul', 874),
 ('folk', 848),
 ('urban', 788),
 ('hop', 778)]

In [22]:
import operator

# create dictionary of word frequency in genre
categories = {}
for genre in data['Genres']:
    genre = genre.replace(",", "")
    words = genre.split(' ')
    count = {}
    for word in words:
        if word in count:
            count[word] += 1
        else:
            count[word] = 1
    word = max(count.items(), key=operator.itemgetter(1))[0]
    if word in categories:
        categories[word] += 1
    else:
        categories[word] = 1

# sort dictionary by frequency to get most common overall genre categories
list({k: v for k, v in sorted(categories.items(), key=lambda item: item[1], reverse=True)}.items())[:15]

[('pop', 1597),
 ('rock', 1164),
 ('hip', 351),
 ('', 323),
 ('soul', 307),
 ('rap', 306),
 ('adult', 281),
 ('country', 252),
 ('new', 172),
 ('disco', 113),
 ('funk', 76),
 ('boy', 71),
 ('bubblegum', 70),
 ('classic', 65),
 ('metal', 52)]